In [1]:
import pandas as pd
import numpy as np
#import gzip
#import json
import sys
#import seaborn as sns
#import ast
import warnings
warnings.filterwarnings('ignore')

#from sklearn.decomposition import PCA
#from scipy.stats import zscore
#from pyod.models.mad import MAD

sys.path.append("../utils/")
from myFunctions import toDommyColumns
#from myFunctions import jsonGzipToDataframe, jsonGzipToDataframe2, toDommyColumns

Guardemos el CSV comprimido dentro de un dataframe Pandas

In [2]:
df = pd.read_csv('../datasource/steam_games_chewed.csv.gz')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32125 entries, 0 to 32124
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    32125 non-null  int64  
 1   publisher     24059 non-null  object 
 2   genres        28844 non-null  object 
 3   app_name      32124 non-null  object 
 4   title         30076 non-null  object 
 5   url           32125 non-null  object 
 6   release_date  30059 non-null  object 
 7   tags          31963 non-null  object 
 8   reviews_url   32125 non-null  object 
 9   specs         31456 non-null  object 
 10  price         30748 non-null  float64
 11  early_access  32125 non-null  bool   
 12  id            32125 non-null  int64  
 13  developer     28828 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 3.2+ MB


#### TAGS 🔎

Los NaN son de tipo float
¿Cuánto representarán del total de valores en la columna 'tags'?

In [4]:
# Crear un nuevo DataFrame solo con las filas que tienen tipo string y no NaN Float en la columna
# 'tags'
dfSinTagsNaN = df[df['tags'].apply(lambda x: isinstance(x, str))]
dfConTagsNaN = df[df['tags'].apply(lambda x: isinstance(x, float))]
print('String son', dfSinTagsNaN.shape[0])
print('NaN son', dfConTagsNaN.shape[0])

String son 31963
NaN son 162


Entonces nos quedearemos con los juegos cuyos 'tags' no sea uno de esos NaN Float

In [5]:
df = dfSinTagsNaN.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31963 entries, 0 to 31962
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    31963 non-null  int64  
 1   publisher     23961 non-null  object 
 2   genres        28820 non-null  object 
 3   app_name      31962 non-null  object 
 4   title         29915 non-null  object 
 5   url           31963 non-null  object 
 6   release_date  29898 non-null  object 
 7   tags          31963 non-null  object 
 8   reviews_url   31963 non-null  object 
 9   specs         31303 non-null  object 
 10  price         30618 non-null  float64
 11  early_access  31963 non-null  bool   
 12  id            31963 non-null  int64  
 13  developer     28693 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 3.2+ MB


Obtengamos las columnas dummies de la columna 'tags'

In [6]:
# Pero antes sustituyamos los NaN de la columna 'tags' por listas vacias
df['tags'] = df['tags'].fillna('[]')

# Obtengamos las columnas dummies de la columna 'tags'
dummyColumnsTags = toDommyColumns(df,'tags')

# Seamos consistentes con los nombres de columnas
dummyColumnsTags.columns = dummyColumnsTags.columns.str.lower().str.replace(' ', '_') 

print(f'Ahora tenemos {dummyColumnsTags.shape[1]} columnas dummies')

Ahora tenemos 339 columnas dummies


Veamos cuáles son las columnas dummies 'tags' más valiosas

In [7]:
# Sumemos los valores de la columna 'tags', los ordenamos en orden descendente, seleccionamos los 20
# valores más altos y luego los normalizamos dividiéndolos por la longitud del DataFrame original
dummyColumnsTags.sum().sort_values(ascending = False).head(20) / len(dummyColumnsTags)

tags_indie               0.551137
tags_action              0.406783
tags_adventure           0.307449
tags_casual              0.304571
tags_simulation          0.241780
tags_strategy            0.236711
tags_rpg                 0.185152
tags_singleplayer        0.135907
tags_free_to_play        0.075181
tags_multiplayer         0.074555
tags_great_soundtrack    0.069893
tags_puzzle              0.066014
tags_early_access        0.060914
tags_2d                  0.060852
tags_atmospheric         0.060476
tags_vr                  0.056190
tags_sports              0.049964
tags_platformer          0.045334
tags_story_rich          0.045177
tags_sci-fi              0.043394
dtype: float64

Nos quedaremos con las columnas cuyo peso sea >= 0,07

In [8]:
tagsColumns = ['tags_indie', 'tags_action', 'tags_adventure', 'tags_casual', 'tags_simulation']
tagsColumns += ['tags_strategy', 'tags_rpg', 'tags_singleplayer', 'tags_free_to_play', 'tags_multiplayer']
dummyTags = dummyColumnsTags[tagsColumns]

In [9]:
dummyTags.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31963 entries, 0 to 31962
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   tags_indie         31963 non-null  int64
 1   tags_action        31963 non-null  int64
 2   tags_adventure     31963 non-null  int64
 3   tags_casual        31963 non-null  int64
 4   tags_simulation    31963 non-null  int64
 5   tags_strategy      31963 non-null  int64
 6   tags_rpg           31963 non-null  int64
 7   tags_singleplayer  31963 non-null  int64
 8   tags_free_to_play  31963 non-null  int64
 9   tags_multiplayer   31963 non-null  int64
dtypes: int64(10)
memory usage: 2.7 MB


In [10]:
dummyTags.head()

,tags_indie,tags_action,tags_adventure,tags_casual,tags_simulation,tags_strategy,tags_rpg,tags_singleplayer,tags_free_to_play,tags_multiplayer
0,1,1,0,1,1,1,0,0,0,0
1,1,0,0,0,0,1,1,0,1,0
2,1,0,0,1,1,0,0,0,1,1
3,0,1,1,1,0,0,0,0,0,0
4,1,1,0,1,0,0,0,0,0,0


In [11]:
# Insertemos las columnas dummies al dataframe original
df = pd.concat([df, dummyTags], axis = 1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31963 entries, 0 to 31962
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         31963 non-null  int64  
 1   publisher          23961 non-null  object 
 2   genres             28820 non-null  object 
 3   app_name           31962 non-null  object 
 4   title              29915 non-null  object 
 5   url                31963 non-null  object 
 6   release_date       29898 non-null  object 
 7   tags               31963 non-null  object 
 8   reviews_url        31963 non-null  object 
 9   specs              31303 non-null  object 
 10  price              30618 non-null  float64
 11  early_access       31963 non-null  bool   
 12  id                 31963 non-null  int64  
 13  developer          28693 non-null  object 
 14  tags_indie         31963 non-null  int64  
 15  tags_action        31963 non-null  int64  
 16  tags_adventure     319

In [13]:
df.head(3)

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,...,tags_indie,tags_action,tags_adventure,tags_casual,tags_simulation,tags_strategy,tags_rpg,tags_singleplayer,tags_free_to_play,tags_multiplayer
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],...,1,1,0,1,1,1,0,0,0,0
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",...,1,0,0,0,0,1,1,0,1,0
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",...,1,0,0,1,1,0,0,0,1,1


#### GENRES 🔎

Los NaN son de tipo float
¿Cuánto representarán del total de valores en la columna 'genres'?

In [14]:
# Crear un nuevo DataFrame solo con las filas que tienen tipo string y no NaN Float en la columna
# 'genres'
dfSinGenresNaN = df[df['genres'].apply(lambda x: isinstance(x, str))]
dfConGenresNaN = df[df['genres'].apply(lambda x: isinstance(x, float))]
print('String son', dfSinGenresNaN.shape[0])
print('NaN son', dfConGenresNaN.shape[0])

String son 28820
NaN son 3143


Entonces nos quedaremos con los juegos cuyos 'genres' no sea uno de esos NaN Float

In [15]:
df = dfSinGenresNaN.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28820 entries, 0 to 28819
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         28820 non-null  int64  
 1   publisher          23910 non-null  object 
 2   genres             28820 non-null  object 
 3   app_name           28819 non-null  object 
 4   title              28819 non-null  object 
 5   url                28820 non-null  object 
 6   release_date       28802 non-null  object 
 7   tags               28820 non-null  object 
 8   reviews_url        28820 non-null  object 
 9   specs              28536 non-null  object 
 10  price              27601 non-null  float64
 11  early_access       28820 non-null  bool   
 12  id                 28820 non-null  int64  
 13  developer          28650 non-null  object 
 14  tags_indie         28820 non-null  int64  
 15  tags_action        28820 non-null  int64  
 16  tags_adventure     288

Obtengamos las columnas dummies de la columna 'genres'

In [16]:
# Obtengamos las columnas dummies de la columna 'genres'
dummyColumnsGenres = toDommyColumns(df, 'genres')

# Seamos consistentes con los nombres de columnas
dummyColumnsGenres.columns = dummyColumnsGenres.columns.str.lower().str.replace(' ', '_') 

In [17]:
print(f'Ahora tenemos {dummyColumnsGenres.shape[1]} columnas dummies')

Ahora tenemos 22 columnas dummies


Veamos cuáles son las columnas dummies 'genres' más valiosas

In [18]:
# Sumemos los valores de la columna 'genres', los ordenamos en orden descendente, seleccionamos los 20
# valores más altos y luego los normalizamos dividiéndolos por la longitud del DataFrame original
dummyColumnsGenres.sum().sort_values(ascending = False).head(20) / len(dummyColumnsGenres)

genres_indie                        0.549757
genres_action                       0.392540
genres_casual                       0.286849
genres_adventure                    0.285704
genres_strategy                     0.241256
genres_simulation                   0.232200
genres_rpg                          0.190042
genres_free_to_play                 0.070472
genres_early_access                 0.050729
genres_sports                       0.043581
genres_massively_multiplayer        0.038446
genres_racing                       0.037578
genres_design_&amp;_illustration    0.015961
genres_utilities                    0.011763
genres_web_publishing               0.009299
genres_animation_&amp;_modeling     0.006350
genres_education                    0.004337
genres_video_production             0.004025
genres_software_training            0.003643
genres_audio_production             0.003227
dtype: float64

Nos quedaremos con las columnas cuyo peso sea >= 0,03

In [19]:
genresColumns = ['genres_indie', 'genres_action', 'genres_casual', 'genres_adventure']
genresColumns += ['genres_strategy', 'genres_simulation', 'genres_rpg', 'genres_free_to_play']
genresColumns += ['genres_early_access', 'genres_sports', 'genres_massively_multiplayer', 'genres_racing']
dummyGenres = dummyColumnsGenres[genresColumns]

In [20]:
dummyGenres

,genres_indie,genres_action,genres_casual,genres_adventure,genres_strategy,genres_simulation,genres_rpg,genres_free_to_play,genres_early_access,genres_sports,genres_massively_multiplayer,genres_racing
0,1,1,1,0,1,1,0,0,0,0,0,0
1,1,0,0,0,1,0,1,1,0,0,0,0
2,1,0,1,0,0,1,0,1,0,1,0,0
3,0,1,1,1,0,0,0,0,0,0,0,0
4,0,1,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
28815,1,1,1,1,0,0,0,0,0,0,0,0
28816,1,0,1,0,1,1,0,0,0,0,0,0
28817,1,0,1,0,1,0,0,0,0,0,0,0
28818,1,0,0,0,0,1,0,0,0,0,0,1


In [21]:
df.shape

(28820, 24)

In [22]:
# Insertemos las columnas dummies al dataframe original
df = pd.concat([df, dummyGenres], axis = 1)

In [23]:
df.head(3)
#df.loc[df['title'] == 'Lost Summoner Kitty']
#df.iloc[:, 7:9]

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,...,genres_casual,genres_adventure,genres_strategy,genres_simulation,genres_rpg,genres_free_to_play,genres_early_access,genres_sports,genres_massively_multiplayer,genres_racing
0,0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],...,1,0,1,1,0,0,0,0,0,0
1,1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",...,0,0,1,0,1,1,0,0,0,0
2,2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",...,1,0,0,1,0,1,0,1,0,0


#### YEAR 🔎

A crear la columna 'year'

In [24]:
# Extraer el año de la columna 'release_date' y crear la columna 'year'
df['year'] = df['release_date'].str.extract(r'(\d{4})')

In [25]:
# Aplicamos una mascara para marcar los valores NaN y los sumamos por columna
df['year'].isna().sum()

119

In [26]:
# Elimina las filas que tienen valor NaN en la columna 'year'
df = df.dropna(subset = ['year']).reset_index(drop = True)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28701 entries, 0 to 28700
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    28701 non-null  int64  
 1   publisher                     23796 non-null  object 
 2   genres                        28701 non-null  object 
 3   app_name                      28700 non-null  object 
 4   title                         28700 non-null  object 
 5   url                           28701 non-null  object 
 6   release_date                  28701 non-null  object 
 7   tags                          28701 non-null  object 
 8   reviews_url                   28701 non-null  object 
 9   specs                         28417 non-null  object 
 10  price                         27594 non-null  float64
 11  early_access                  28701 non-null  bool   
 12  id                            28701 non-null  int64  
 13  d

In [28]:
# Convertimos el año a tipo entero
df['year'] = df['year'].astype(int)

Ok, está quedando mejor. Parece que las columnas 'url' y 'reviews_url' sobran porque no aportan nada
al análisis, vamos a quitarlas


In [29]:
# Eliminar las columnas 'url' y 'reviews_url'
df = df.drop(['url', 'reviews_url'], axis=1)

Ya conseguimos el año a partir de la columna 'release_date', vamos a borrarla. Es redundante tener a
'genres' y 'tags'<br>porque las desplegamos como dummies, tambien a borrarlas. 

In [30]:
# Eliminar las columnas 'release_date', 'genres' y 'tags'
df = df.drop(['release_date', 'genres', 'tags'], axis=1)

In [31]:
df.head(3)

,Unnamed: 0,publisher,app_name,title,specs,price,early_access,id,developer,tags_indie,...,genres_adventure,genres_strategy,genres_simulation,genres_rpg,genres_free_to_play,genres_early_access,genres_sports,genres_massively_multiplayer,genres_racing,year
0,0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,['Single-player'],4.99,False,761140,Kotoshiro,1,...,0,1,1,0,0,0,0,0,0,2018
1,1,"Making Fun, Inc.",Ironbound,Ironbound,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,643980,Secret Level SRL,1,...,0,1,0,1,1,0,0,0,0,2018
2,2,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,670290,Poolians.com,1,...,0,0,1,0,1,0,1,0,0,2017


Los True y False de la columna 'early_access' mejor los cambiamos a unos y ceros respectivamente

In [32]:
# Cambiar la columna 'early_access' a tipo entero en el DataFrame 'df'
df['early_access'] = df['early_access'].astype(int)

In [33]:
# Ver el tipo de dato de la columna 'tags' de la primera fila
type(df.iloc[0]['early_access'])

numpy.int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28701 entries, 0 to 28700
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    28701 non-null  int64  
 1   publisher                     23796 non-null  object 
 2   app_name                      28700 non-null  object 
 3   title                         28700 non-null  object 
 4   specs                         28417 non-null  object 
 5   price                         27594 non-null  float64
 6   early_access                  28701 non-null  int64  
 7   id                            28701 non-null  int64  
 8   developer                     28531 non-null  object 
 9   tags_indie                    28701 non-null  int64  
 10  tags_action                   28701 non-null  int64  
 11  tags_adventure                28701 non-null  int64  
 12  tags_casual                   28701 non-null  int64  
 13  t

In [35]:
# Remove 'playtime_2weeks' column
df = df.drop(['early_access'], axis = 1)

#### Save 💾

En este punto el dataframe df esta listo para ser la fuente de datos de tres de las cinco funciones que responderan<br>desde la API. Siendo asi, tomaremos una instantanea del estado actual de los datos y los persistiremos en un CSV. 

In [36]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_csv('../datasource/render/steam_games_4Api.csv.gz', compression='gzip', index = False)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28701 entries, 0 to 28700
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    28701 non-null  int64  
 1   publisher                     23796 non-null  object 
 2   app_name                      28700 non-null  object 
 3   title                         28700 non-null  object 
 4   specs                         28417 non-null  object 
 5   price                         27594 non-null  float64
 6   id                            28701 non-null  int64  
 7   developer                     28531 non-null  object 
 8   tags_indie                    28701 non-null  int64  
 9   tags_action                   28701 non-null  int64  
 10  tags_adventure                28701 non-null  int64  
 11  tags_casual                   28701 non-null  int64  
 12  tags_simulation               28701 non-null  int64  
 13  t

#### LA CONSULTA PARA LA API

In [38]:
def developer(desarrollador: str):
    '''Devuelve la cantidad de items y porcentaje de contenido Free por año según empresa
       desarrolladora
    '''
    # Filtrar el DataFrame por desarrollador
    dfFiltrado = df[df['developer'] == desarrollador]

    # Contar la cantidad de items por año
    cantidadItems = dfFiltrado.groupby('year').size().reset_index(name = 'Cantidad de Items')

    # Calcular el porcentaje de juegos gratuitos por año
    dfFiltrado['free'] = (dfFiltrado['tags_free_to_play'] == 1) | (dfFiltrado['price'] == 0)
    contenidoFree = dfFiltrado.groupby('year')['free'].mean().reset_index(name = 'Contenido Free')

    # Combinar los resultados en un nuevo DataFrame
    return cantidadItems.merge(contenidoFree, on = 'year').to_dict('records')

In [39]:
developer('Valve')

[{'year': 1998, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 1999, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 2000, 'Cantidad de Items': 2, 'Contenido Free': 0.0},
 {'year': 2001, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 2003, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 2004, 'Cantidad de Items': 5, 'Contenido Free': 0.0},
 {'year': 2005, 'Cantidad de Items': 1, 'Contenido Free': 1.0},
 {'year': 2006, 'Cantidad de Items': 2, 'Contenido Free': 0.0},
 {'year': 2007, 'Cantidad de Items': 3, 'Contenido Free': 0.3333333333333333},
 {'year': 2008, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 2009, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 2010, 'Cantidad de Items': 2, 'Contenido Free': 0.5},
 {'year': 2011, 'Cantidad de Items': 1, 'Contenido Free': 0.0},
 {'year': 2012, 'Cantidad de Items': 2, 'Contenido Free': 0.5},
 {'year': 2014, 'Cantidad de Items': 7, 'Contenido Free': 0.14285714285714285},
 {'year':

Busquemos pistas sobre la presencia de outliers con un histograma

In [40]:
# Create a histogram using Seaborn
#g = sns.histplot(data = df, x = 'price')
# Add labels
#g.set_xlabel('Total price per game')

En el histograma podemos ver que los datos se concentran por debajo de los 50$ aproximadamente

Busquemos pistas sobre la presencia de outliers con un diagrama de caja

In [41]:
# Create a box plot
g = sns.boxplot(data = df, x = 'price')

# Add a title and change xlabel
g.set_title('Box Plot of Total')
g.set_xlabel('Total price per game')

NameError: name 'sns' is not defined

En el gráfico de caja podemos ver claramente la presencia de outliers en la columna ‘price’,

Ataquemos ahora con Z-Score

In [ ]:
# Calculate z-score for each data point and compute its absolute value
z_scores = zscore(df['price'])
abs_z_scores = np.abs(z_scores)

# Select the outliers using a threshold of 3
outliers = df[abs_z_scores > 3]
outliers.head()

In [ ]:
# Obtain number of outliers
print(f'Number of outliers: {len(outliers)}')

Pero no podemos fiarnos de estos 287 porque el método z-score sólo es apropiada para distribuciones normales. Como vimos en el histograma, los datos para la variable ‘price’ están sesgados hacia la derecha por lo que debemos afinar la puntería.

Busquemos pistas sobre la presencia de outliers con MAD-Z-Score

In [ ]:
# Set threshold to 3.5
mad = MAD(threshold = 3.5)

# Convert the 'total' column into a 2D numpy array
priceReshaped = df['price'].values.reshape(-1, 1)

# Generate inline and outlier labels
labels = mad.fit(priceReshaped).labels_
labels

In [ ]:
# Obtain number of outliers
print(f'Number of outliers: {labels.sum()}')